# PN Ou 5: Inspect original files

In [24]:
from pathlib import Path
from astropy.io import fits
from astropy.table import Table
import astropy.table

In [2]:
dpath = Path("../data/originals/")

Look and see what sort of files we have:

In [51]:
data = []
hdrs = []
kwds = ["MEZMODE", "DATE-OBS", "FILTER", "EXPTIME", "RA", "DEC", "PA", "CCDTYPE", "CCDSUM"]
for _file in sorted(dpath.glob("*.fits")):
    hdu = fits.open(_file)[0]
    thisdata = {"File": _file.stem}
    for k in kwds:
        thisdata[k] = hdu.header.get(k)
    if "DISPAXIS" in hdu.header:
        dlam = min(hdu.header["CDELT1"], hdu.header["CDELT2"])
        lam0 = max(hdu.header["CRVAL1"], hdu.header["CRVAL2"])
        nbin = int(thisdata["CCDSUM"][0])
        thisdata["DV"] = 3e5 * dlam / lam0 / nbin
    data.append(thisdata)
    hdrs.append(hdu.header)
tab = Table(rows=data)
tab.show_in_notebook()

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

So we have 2017 data with 70 micron slit and 2x2 binning, and then 2018, 2019 data with 150 micron slit and 3x3 binning.

Select the image+slit or slit+image files that we will need to do astrometry of

In [48]:
hdrs[26]

SIMPLE  =                    T / Fits standard                                  
BITPIX  =                  -32 / Bits per pixel                                 
NAXIS   =                    2 / Number of axes                                 
NAXIS1  =                  667 / Axis length                                    
NAXIS2  =                  655 / Axis length                                    
EXTEND  =                    F / File may contain extensions                    
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
DATE    = '2019-02-05T08:29:33' / Date FITS file was generated                  
IRAF-TLM= '2019-02-05T08:29:33' / Time of last modification                     
OBJECT  = 'espectro Ha Ou5'    / Name of the object observed                    
COMMENT and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H   
EXPTIME =               1800.0 / Integration Time, sec.                         
CCDTEMP =               -110

In [45]:
2 * 3e5 * hdrs[2].get("cdelt2") / hdrs[2].get("crval2")

5.178938288213032

In [4]:
m = ["slit" in _ for _ in tab["MEZMODE"]]
tab[m]

File,MEZMODE,DATE-OBS,FILTER,EXPTIME,RA,DEC,PA,CCDTYPE,CCDSUM
str18,str14,str10,str8,float64,str11,str12,str15,str8,str3
crN10035_b,image_slit_70,2017-08-29,Ha 90A,100.0,21:15:04.4,43:46:39.0,"8.63 , -351.37",E2V-4240,2 2
crN10042_b,image_slit_70,2017-08-29,Ha 90A,100.0,21:15:02.2,43:46:58.0,"8.63 , -351.37",E2V-4240,2 2
crN10047o,image_slit_70,2017-08-29,OIII 60A,100.0,21:15:02.5,43:47:23.0,"8.63 , -351.37",E2V-4240,2 2
crN20001_b,slit_image_70,2017-08-30,NII,100.0,21:15:02.6,43:45:32.0,"360.00 , -0.00",E2V-4240,2 2
crN20008_b,slit_image_70,2017-08-30,OIII 60A,100.0,21:15:04.1,43:45:25.0,"360.00 , -0.00",E2V-4240,2 2
crN20011_b,slit_image_70,2017-08-30,Ha 90A,100.0,21:15:03.3,43:45:22.0,"360.00 , -0.00",E2V-4240,2 2
crspm0020o_b,image_slit_150,2018-05-02,Ha 90A,100.0,21:15:03.9,43:49:3.0,"359.90 , -0.10",E2V-4240,3 3
crspm0025o_b,slit_image_150,2018-05-02,OIII 60A,100.0,21:15:06.5,43:48:53.0,"359.90 , -0.10",E2V-4240,3 3
crspm0047o_b,image_slit_150,2018-05-03,Ha 90A,100.0,21:15:04.8,43:49:47.0,"359.98 , -0.02",E2V-4240,3 3


Write out a list of all the Image+slit files

In [5]:
listfile = dpath.parent / "image-list.dat"
listfile.write_text("\n".join(tab[m]["File"]))
listfile

PosixPath('../data/image-list.dat')

Check that it worked:

In [6]:
listfile.read_text().splitlines()

['crN10035_b',
 'crN10042_b',
 'crN10047o',
 'crN20001_b',
 'crN20008_b',
 'crN20011_b',
 'crspm0020o_b',
 'crspm0025o_b',
 'crspm0047o_b',
 'crspm0052o_b',
 'crspm0053o_b',
 'crspm0058o_b',
 'crspm0109o_b',
 'crspm0210o_b',
 'crspm0214o_b',
 'crspm0431o_b',
 'crspm0439o_b',
 'crspm0600o_b']

## Find the HEALpix coordinates of our source

In [7]:
from astropy.coordinates import SkyCoord, ICRS
import astropy.units as u

All the positions should be about the same, so we just use the first one.

In [8]:
c = SkyCoord(tab[0]["RA"], tab[0]["DEC"], unit=(u.hourangle, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    (318.76833333, 43.7775)>

In [9]:
from astropy_healpix import HEALPix

In order to find which data files to download from http://data.astrometry.net/5000/, we need to translate the celestial coordinate to HEALpix index numbers:

In [10]:
hp_2 = HEALPix(nside=2, order="nested", frame=ICRS())
hp_1 = HEALPix(nside=1, order="nested", frame=ICRS())

Levels 0 to 4 use the `nside=2` tiles. 

In [11]:
hp_2.cone_search_skycoord(c, radius=5 * u.arcminute)

array([13])

So that means `index500[0-4]-13.fits`

In [12]:
hp_1.cone_search_skycoord(c, radius=5 * u.arcminute)

array([3])

So that means `index500[5-7]-03.fits`

In [13]:
hp_2.cone_search_lonlat(300 * u.deg, 50 * u.deg, 0.1 * u.deg)

array([14])

## Look at the HEALpix data files

Something isn't right.  I got the 13 series but the program complains that the coordinates are not contained in the tile.

In [14]:
hdulist = fits.open(dpath.parent / "astrometry-net" / "index-5004-13.fits")
hdulist.info()

Filename: ../data/astrometry-net/index-5004-13.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     109   ()      
  1                1 BinTableHDU     12   774400R x 1C   [16A]   
  2                1 BinTableHDU    138   0R x 1C   [0A]   
  3                1 BinTableHDU     16   32768R x 1C   [4A]   
  4                1 BinTableHDU     21   32767R x 1C   [2A]   
  5                1 BinTableHDU     29   9R x 1C   [8A]   
  6                1 BinTableHDU     14   774400R x 1C   [8A]   
  7                1 BinTableHDU    111   0R x 1C   [0A]   
  8                1 BinTableHDU     16   32768R x 1C   [4A]   
  9                1 BinTableHDU     21   32767R x 1C   [4A]   
 10                1 BinTableHDU     28   7R x 1C   [8A]   
 11                1 BinTableHDU     14   484000R x 1C   [12A]   
 12                1 BinTableHDU     12   484000R x 1C   [1B]   
 13                1 BinTableHDU     37   484000R x 13C   [1K, 1E, 1E, 1E, 1E

Looks like HDU 13 has the original table of stars:

In [15]:
hdulist[13].header

XTENSION= 'BINTABLE' / FITS Binary Table Extension                              
BITPIX  =                    8 / 8-bits character format                        
NAXIS   =                    2 / Tables are 2-D char. array                     
NAXIS1  =                   64 / Bytes in row                                   
NAXIS2  =               484000 / no comment                                     
PCOUNT  =                    0 / Parameter count always 0                       
GCOUNT  =                    1 / Group count always 1                           
TFIELDS =                   13 / No. of col in table                            
TFORM1  = '1K      ' / Format of field                                          
TTYPE1  = 'source_id' / Field label                                             
TFORM2  = '1E      ' / Format of field                                          
TTYPE2  = 'phot_g_mean_mag' / Field label                                       
TFORM3  = '1E      ' / Forma

In [16]:
tstars = Table.read(hdulist[13])

In [17]:
df = tstars.to_pandas()

In [18]:
df[["ra", "dec"]].describe()

,ra,dec
count,484000.000000,484000.000000
mean,289.941766,42.040913
std,10.589280,9.873081
min,270.000057,19.512950
25%,281.590965,34.682751
50%,290.029967,41.809240
75%,297.865546,49.214437
max,314.928236,66.385848


So no wonder that is not working.  I want (318.6, 43.7) but this has an RA range of 270 to 315

In [19]:
tstars2 = Table.read(fits.open(dpath.parent / "astrometry-net" / "index-5004-14.fits")[13])

In [20]:
df2 = tstars2.to_pandas()

In [21]:
df2[["ra", "dec"]].describe()

,ra,dec
count,484000.000000,484000.000000
mean,340.058278,42.041017
std,10.589576,9.873239
min,315.036999,19.501184
25%,332.137109,34.682309
50%,339.974504,41.808876
75%,348.411307,49.212631
max,359.999834,66.401812


So, it turns out that tile 14 is what I needed, not 13.